In [1]:
import numpy as np

from sklearn.metrics import accuracy_score
from keras.datasets import reuters
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
# parameters for data load
num_words = 30000
maxlen = 50
test_split = 0.3

In [3]:
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words = num_words, maxlen = maxlen, test_split = test_split)

In [4]:
# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, padding = 'post')
X_test = pad_sequences(X_test, padding = 'post')

X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

y_data = np.concatenate((y_train, y_test))
y_data = to_categorical(y_data)
y_train = y_data[:1395]
y_test = y_data[1395:]

## 1. Deep RNN
- RNNs can be made deep, with multiple layers, like CNNs or MLPs
- Beware that RNNs take long to train compared to CNNs

In [7]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

In [10]:
def deep_lstm():
    model = Sequential()
    model.add(LSTM(20, input_shape = (49,1), return_sequences = True))
    model.add(LSTM(20, return_sequences = True))
    model.add(LSTM(20, return_sequences = True))
    model.add(LSTM(20, return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [11]:
model = KerasClassifier(build_fn = deep_lstm, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)

Epoch 1/200
1395/1395 [==============================] - 9s - loss: 3.2011 - acc: 0.6616     
Epoch 2/200
1395/1395 [==============================] - 5s - loss: 1.8565 - acc: 0.7147     
Epoch 3/200
1395/1395 [==============================] - 5s - loss: 1.2749 - acc: 0.7147     
Epoch 4/200
1395/1395 [==============================] - 5s - loss: 1.2051 - acc: 0.7147     
Epoch 5/200
1395/1395 [==============================] - 5s - loss: 1.1878 - acc: 0.7147     
Epoch 6/200
1395/1395 [==============================] - 6s - loss: 1.1791 - acc: 0.7147     
Epoch 7/200
1395/1395 [==============================] - 5s - loss: 1.1733 - acc: 0.7147     
Epoch 8/200
1395/1395 [==============================] - 6s - loss: 1.1703 - acc: 0.7147     
Epoch 9/200
1395/1395 [==============================] - 5s - loss: 1.1667 - acc: 0.7147     
Epoch 10/200
1395/1395 [==============================] - 5s - loss: 1.1642 - acc: 0.7147     
Epoch 11/200
1395/1395 [==============================] - 5

1395/1395 [==============================] - 5s - loss: 0.5638 - acc: 0.8502     
Epoch 172/200
1395/1395 [==============================] - 5s - loss: 0.5553 - acc: 0.8509     
Epoch 173/200
1395/1395 [==============================] - 5s - loss: 0.5632 - acc: 0.8545     
Epoch 174/200
1395/1395 [==============================] - 5s - loss: 0.5573 - acc: 0.8595     - ETA: 2s - loss: 0.5214 -
Epoch 175/200
1395/1395 [==============================] - 6s - loss: 0.5666 - acc: 0.8502     
Epoch 176/200
1395/1395 [==============================] - 5s - loss: 0.5537 - acc: 0.8545     
Epoch 177/200
1395/1395 [==============================] - 5s - loss: 0.5515 - acc: 0.8566     
Epoch 178/200
1395/1395 [==============================] - 5s - loss: 0.5381 - acc: 0.8645     
Epoch 179/200
1395/1395 [==============================] - 5s - loss: 0.5335 - acc: 0.8645     
Epoch 180/200
1395/1395 [==============================] - 5s - loss: 0.5477 - acc: 0.8538     
Epoch 181/200
1395/1395 [===

In [12]:
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

599/599 [==============================] - 0s     


In [13]:
print(accuracy_score(y_pred, y_test_))

0.816360601002


## 2. Bidirectional RNN
- Bidirectional RNNs consider not only one-way influence of sequence, but also the other way
- Actually, they can be thought as building two separate RNNs, and merging them\
<br>
<img src="http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/09/bidirectional-rnn.png" style="width: 400px"/>
</br>

In [14]:
from keras.layers import Bidirectional

In [21]:
def bidirectional_lstm():
    model = Sequential()
    model.add(Bidirectional(LSTM(20, return_sequences = False), input_shape = (49,1)))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [22]:
model = KerasClassifier(build_fn = bidirectional_lstm, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)

Epoch 1/200
1395/1395 [==============================] - 4s - loss: 3.4012 - acc: 0.4896     
Epoch 2/200
1395/1395 [==============================] - 2s - loss: 2.1016 - acc: 0.7147     
Epoch 3/200
1395/1395 [==============================] - 2s - loss: 1.2732 - acc: 0.7147     
Epoch 4/200
1395/1395 [==============================] - 2s - loss: 1.1607 - acc: 0.7147     
Epoch 5/200
1395/1395 [==============================] - 2s - loss: 1.1129 - acc: 0.7147     
Epoch 6/200
1395/1395 [==============================] - 2s - loss: 1.0771 - acc: 0.7147     
Epoch 7/200
1395/1395 [==============================] - 2s - loss: 1.0392 - acc: 0.7147     
Epoch 8/200
1395/1395 [==============================] - 2s - loss: 1.0013 - acc: 0.7140     
Epoch 9/200
1395/1395 [==============================] - 2s - loss: 0.9633 - acc: 0.7412     
Epoch 10/200
1395/1395 [==============================] - 2s - loss: 0.9295 - acc: 0.7577     - ETA: 0s - loss: 0.9232 - acc:
Epoch 11/200
1395/1395 [====

1395/1395 [==============================] - 2s - loss: 0.4135 - acc: 0.8918     
Epoch 172/200
1395/1395 [==============================] - 2s - loss: 0.4295 - acc: 0.8875     
Epoch 173/200
1395/1395 [==============================] - 2s - loss: 0.4106 - acc: 0.8925     
Epoch 174/200
1395/1395 [==============================] - 2s - loss: 0.4029 - acc: 0.8946     
Epoch 175/200
1395/1395 [==============================] - 2s - loss: 0.4024 - acc: 0.8946     
Epoch 176/200
1395/1395 [==============================] - 2s - loss: 0.4061 - acc: 0.8932     
Epoch 177/200
1395/1395 [==============================] - 3s - loss: 0.3943 - acc: 0.8989     
Epoch 178/200
1395/1395 [==============================] - 2s - loss: 0.3926 - acc: 0.8975     
Epoch 179/200
1395/1395 [==============================] - 2s - loss: 0.3969 - acc: 0.9018     
Epoch 180/200
1395/1395 [==============================] - 2s - loss: 0.3936 - acc: 0.8982     
Epoch 181/200
1395/1395 [=============================

In [23]:
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

500/599 [========================>.....] - ETA: 0s

In [24]:
print(accuracy_score(y_pred, y_test_))

0.841402337229


## 3. Deep Bidirectional RNN
- Bidirectional RNNs can be stacked

<img src="http://www.wildml.com/wp-content/uploads/2015/09/Screen-Shot-2015-09-16-at-2.21.51-PM-272x300.png" style="width: 300px"/>

In [27]:
def deep_bidirectional_lstm():
    model = Sequential()
    model.add(Bidirectional(LSTM(10, return_sequences = True), input_shape = (49,1)))
    model.add(Bidirectional(LSTM(10, return_sequences = True)))
    model.add(Bidirectional(LSTM(10, return_sequences = True)))
    model.add(Bidirectional(LSTM(10, return_sequences = False)))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [28]:
model = KerasClassifier(build_fn = deep_bidirectional_lstm, epochs = 200, batch_size = 50, verbose = 1)
model.fit(X_train, y_train)

Epoch 1/200
1395/1395 [==============================] - 15s - loss: 3.3987 - acc: 0.6910    
Epoch 2/200
1395/1395 [==============================] - 11s - loss: 2.3765 - acc: 0.7147    
Epoch 3/200
1395/1395 [==============================] - 11s - loss: 1.5303 - acc: 0.7147    
Epoch 4/200
1395/1395 [==============================] - 11s - loss: 1.2363 - acc: 0.7147    
Epoch 5/200
1395/1395 [==============================] - 11s - loss: 1.1987 - acc: 0.7147    
Epoch 6/200
1395/1395 [==============================] - 11s - loss: 1.1844 - acc: 0.7147    
Epoch 7/200
1395/1395 [==============================] - 11s - loss: 1.1776 - acc: 0.7147    
Epoch 8/200
1395/1395 [==============================] - 11s - loss: 1.1694 - acc: 0.7147    
Epoch 9/200
1395/1395 [==============================] - 11s - loss: 1.1602 - acc: 0.7147    
Epoch 10/200
1395/1395 [==============================] - 11s - loss: 1.1476 - acc: 0.7147    
Epoch 11/200
1395/1395 [==============================] - 1

1395/1395 [==============================] - 10s - loss: 0.5985 - acc: 0.8409    
Epoch 173/200
1395/1395 [==============================] - 11s - loss: 0.6027 - acc: 0.8416    
Epoch 174/200
1395/1395 [==============================] - 10s - loss: 0.6140 - acc: 0.8401    
Epoch 175/200
1395/1395 [==============================] - 10s - loss: 0.6485 - acc: 0.8315    
Epoch 176/200
1395/1395 [==============================] - 10s - loss: 0.6224 - acc: 0.8351    
Epoch 177/200
1395/1395 [==============================] - 10s - loss: 0.6042 - acc: 0.8452    
Epoch 178/200
1395/1395 [==============================] - 10s - loss: 0.6007 - acc: 0.8409    
Epoch 179/200
1395/1395 [==============================] - 11s - loss: 0.5930 - acc: 0.8473    
Epoch 180/200
1395/1395 [==============================] - 11s - loss: 0.5887 - acc: 0.8495    
Epoch 181/200
1395/1395 [==============================] - 11s - loss: 0.5975 - acc: 0.8473    
Epoch 182/200
1395/1395 [=============================

In [29]:
y_pred = model.predict(X_test)
y_test_ = np.argmax(y_test, axis = 1)

599/599 [==============================] - 1s     


In [30]:
print(accuracy_score(y_pred, y_test_))

0.824707846411
